In [8]:
# @title Import Libraries
import pandas as pd
import tracemalloc
import time
import psutil
from realtabformer import REaLTabFormer
from transformers import GPT2Config
from src.data_processing import csv_data_split

In [9]:
train_data, test_data, sample_data = csv_data_split("../data/breast-cancer-wisconsin.csv")

In [10]:
# Initialize an empty DataFrame to store results
results = pd.DataFrame(columns=["Model", "Time (s)", "Memory Current (MB)", "Memory Peak (MB)"])

In [36]:
results["System Memory Usage (MB)"] = 0
results

,Model,Time (s),Memory Current (MB),Memory Peak (MB),Layers,Heads,Embedding Dim,DS Size,System Memory Usage (MB)
0,small_model,566.675813,42.637881,47.304494,4,8,512,273,0
1,regular_model,1790.282550,1.735398,2.291640,6,12,768,273,0
2,large_model,4983.763235,1.624654,2.558731,12,12,768,273,0


In [42]:
new_results = pd.DataFrame(columns=["Model", "Time (s)", "Memory Current (MB)", "Memory Peak (MB)", "System Memory Usage (MB)"])
new_results

,Model,Time (s),Memory Current (MB),Memory Peak (MB),System Memory Usage (MB)


In [43]:
def fit_and_track_memory(model, data, model_name):
    tracemalloc.start()
    start_time = time.time()

    process = psutil.Process()
    initial_memory = process.memory_info().rss

    model.fit(data)

    end_time = time.time()
    elapsed_time = end_time - start_time

    final_memory = process.memory_info().rss
    current, peak = tracemalloc.get_traced_memory()

    print(f"Model: {model_name}")
    print(f"System Memory Usage: {((final_memory - initial_memory) / 1024**2):.2f} MB")
    print(f"Python Current Memory: {current / 1024**2:.2f} MB")
    print(f"Python Peak Memory: {peak / 1024**2:.2f} MB")

    tracemalloc.stop()

    new_results.loc[len(results)] = [
        model_name,
        elapsed_time,
        current / 1024**2,
        peak / 1024**2,
        (final_memory - initial_memory) / 1024**2
    ]

    model.save(f"../models/{model_name}")

In [43]:
def fit_and_track(model, data, model_name):
    start_time = time.time()
    model.fit(data)
    end_time = time.time()
    elapsed_time = end_time - start_time


    new_results.loc[len(results)] = [
        model_name,
        elapsed_time
    ]

    model.save(f"../models/{model_name}")

In [44]:
config_small = GPT2Config(
    n_embd=512,
    n_layer=4,
    n_head=8
)

rtf_model_small = REaLTabFormer(
    model_type="tabular",
    tabular_config=config_small,
    batch_size=16,
    epochs=75,
    mask_rate=0.1,
)

rtf_model_reg = REaLTabFormer(
    model_type="tabular",
    batch_size=16,
    epochs=75,
    mask_rate=0.1,
)

# rtf_model_large = REaLTabFormer(
#     model_type="tabular",
#     tabular_config=config_large,
#     batch_size=16,
#     epochs=75,
#     mask_rate=0.3,
# )

# Fit models and track performance
fit_and_track_memory(rtf_model_small, train_data, "small_model_full")
fit_and_track_memory(rtf_model_reg, train_data, "regular_model_full")
# fit_and_track_memory(rtf_model_large, sample_df, "large_model")

# Display the results
print(results)

Computing the sensitivity threshold...
Using parallel computation!!!


/Users/sebastian/PycharmProjects/realtabformer-compression/venv/lib/python3.11/site-packages/realtabformer/realtabformer.py:288: UserWarning: The `bos_token_id`, `eos_token_id`, and `vocab_size` attributes will                     be replaced when the `.fit` method is run.
  warnings.warn(
/Users/sebastian/PycharmProjects/realtabformer-compression/venv/lib/python3.11/site-packages/realtabformer/realtabformer.py:77: UserWarning: The device=cuda is not available, using device=cpu instead.
  warnings.warn(
/Users/sebastian/PycharmProjects/realtabformer-compression/venv/lib/python3.11/site-packages/realtabformer/realtabformer.py:597: UserWarning: qt_interval adjusted from 100 to 4...
  warnings.warn(


Bootstrap round:   0%|          | 0/500 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    500.000000
mean       0.004391
std        0.013035
min       -0.021667
25%       -0.005000
50%        0.002500
75%        0.011806
max        0.057778
dtype: float64
Sensitivity threshold: 0.02722222222222222 qt_max: 0.05


Map:   0%|          | 0/546 [00:00<?, ? examples/s]

...

  0%|          | 0/270 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 384 samples generated. Sampling efficiency is: 100.0000%
Saving not-best model...
Critic round: 60,                     sensitivity_threshold: 0.028333333333333332,                         val_sensitivity: 0.02907407407407407,                             val_sensitivities: [0.004444444444444444, 0.052777777777777785, 0.018333333333333333, 0.019444444444444445, 0.03833333333333333, 0.04, 0.009444444444444445, 0.058333333333333334, 0.02888888888888889, 0.023333333333333334, 0.05666666666666666, 0.013333333333333334, 0.013333333333333332, 0.042777777777777776, 0.01666666666666667]


Step,Training Loss


  0%|          | 0/270 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 384 samples generated. Sampling efficiency is: 100.0000%
Critic round: 65,                     sensitivity_threshold: 0.028333333333333332,                         val_sensitivity: 0.04222222222222223,                             val_sensitivities: [0.004999999999999999, 0.02611111111111111, 0.02277777777777778, 0.019999999999999997, 0.034999999999999996, 0.04166666666666667, 0.01388888888888889, 0.05333333333333333, 0.03277777777777778, 0.018333333333333333, 0.12833333333333335, 0.03944444444444445, 0.05444444444444445, 0.09888888888888889, 0.043333333333333335]
Stopping training, no improvement in critic...
Model: regular_model_full
System Memory Usage: -328.23 MB
Python Current Memory: 1.09 MB
Python Peak Memory: 4.51 MB
Copying artefacts from: best-disc-model
Copying artefacts from: mean-best-disc-model
Copying artefacts from: not-best-disc-model
Copying artefacts from: last-epoch-model
           Model     Time (s)  Memory Current (MB)  Mem

In [57]:
new_results.loc[2] = ['small_model_full',1101.653454,23.20,26.57,-104.24]
new_results

,Model,Time (s),Memory Current (MB),Memory Peak (MB),System Memory Usage (MB)
3,regular_model_full,2710.346546,1.090179,4.505012,-328.234375
2,small_model_full,1101.653454,23.200000,26.570000,-104.240000


In [ ]:
2710.346546

In [16]:
layers = [4,6,12]
heads = [8,12,12]
n_embd = [512,768,768]
train_length = [len(sample_df),len(sample_df),len(sample_df)]

results['Layers'] = layers
results['Heads'] = heads
results['Embedding Dim'] = n_embd
results['DS Size'] = train_length


good_results = results[['Model','Time (s)','Layers','Heads','Embedding Dim','DS Size']]
good_results

,Model,Time (s),Layers,Heads,Embedding Dim,DS Size
0,small_model,566.675813,4,8,512,273
1,regular_model,1790.282550,6,12,768,273
2,large_model,4983.763235,12,12,768,273


In [18]:
good_results.to_csv('logs.csv')